# Creating and Backtesting MACD Strategies

## Getting the Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn")

In [ ]:
data = pd.read_csv("eurusd.csv", parse_dates = ["Date"], index_col = "Date")

In [ ]:
data

In [ ]:
data.info()

In [ ]:
data.plot(figsize = (12, 8), title = "EUR/USD", fontsize = 12)
plt.show()

## Defining an MACD Strategy (Part 1)

In [ ]:
data

In [ ]:
ema_s = 12 # EMA Short
ema_l = 26 # EMA Long

In [ ]:
data["EMA_S"] = data.price.ewm(span = ema_s, min_periods = ema_s).mean() 

In [ ]:
data["EMA_L"] = data.price.ewm(span = ema_l, min_periods = ema_l).mean() 

In [ ]:
data

In [ ]:
data.loc["2016", ["price", "EMA_S", "EMA_L"]].plot(figsize = (12, 8))
plt.show()

In [ ]:
data["MACD"] = data.EMA_S - data.EMA_L

In [ ]:
data

In [ ]:
data.loc["2016", ["EMA_S", "EMA_L", "MACD"]].plot(figsize = (12, 8), secondary_y = "MACD")
plt.show()

In [ ]:
signal_mw = 9 # Moving Window for Signal Line

In [ ]:
data["MACD_Signal"] = data.MACD.ewm(span = signal_mw, min_periods = signal_mw).mean() 

In [ ]:
data

In [ ]:
data.loc["2016"]

In [ ]:
data.loc["2016", ["MACD", "MACD_Signal"]].plot(figsize = (12, 8))
plt.show()

## Defining an MACD Strategy (Part 2)

In [ ]:
data["position"] = np.where(data.MACD - data.MACD_Signal > 0, 1, -1)

In [ ]:
data

In [ ]:
data.loc[:, ["MACD", "MACD_Signal", "position"]].plot(figsize = (12, 8), fontsize = 12, 
                                                      secondary_y = "position", title = "EUR/USD")
plt.show()

In [ ]:
data.loc["2016", ["MACD", "MACD_Signal", "position"]].plot(figsize = (12, 8), fontsize = 12,
                                                           secondary_y = "position", title = "EUR/USD")
plt.show()

## Vectorized Strategy Backtesting

In [ ]:
data

In [ ]:
data["returns"] = np.log(data.price.div(data.price.shift(1)))

In [ ]:
data.dropna(inplace = True)

In [ ]:
data

In [ ]:
data["strategy"] = data.position.shift(1) * data["returns"]

In [ ]:
data

In [ ]:
data.dropna(inplace = True)

In [ ]:
ptc = 0.00007

In [ ]:
data["trades"] = data.position.diff().fillna(0).abs()

In [ ]:
data.trades.value_counts()

In [ ]:
data["strategy_net"] = data.strategy - data.trades * ptc

In [ ]:
data["creturns"] = data["returns"].cumsum().apply(np.exp)
data["cstrategy"] = data["strategy"].cumsum().apply(np.exp)
data["cstrategy_net"] = data["strategy_net"].cumsum().apply(np.exp)

In [ ]:
data

In [ ]:
data[["creturns", "cstrategy", "cstrategy_net"]].plot(figsize = (12 , 8))
plt.show()

In [ ]:
data.loc["2020", ["creturns", "cstrategy", "cstrategy_net"]].plot(figsize = (12 , 8))
plt.show()

## Using the MACDBacktester Class

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import brute
plt.style.use("seaborn")

In [ ]:
pd.read_csv("forex_pairs.csv")

In [ ]:
from MACDBacktester import MACDBacktester as MACD

In [ ]:
ptc = 0.00007

In [ ]:
tester = MACD("EURUSD=X", EMA_S = 12, EMA_L = 26, signal_mw = 9,
              start = "2004-01-01", end = "2020-06-30", tc = ptc)

In [ ]:
tester

In [ ]:
tester.data

In [ ]:
tester.test_strategy()

In [ ]:
tester.plot_results()

In [ ]:
tester.results

In [ ]:
tester.optimize_parameters((5, 20, 1), (21, 50, 1), (5, 20, 1))

In [ ]:
tester.plot_results()

__AUD / EUR__

In [ ]:
tester = MACD("AUDEUR=X", EMA_S = 12, EMA_L = 26, signal_mw = 9, 
              start = "2004-01-01", end = "2020-06-30", tc = 0)

In [ ]:
tester.test_strategy()

In [ ]:
tester.plot_results()

In [ ]:
tester.optimize_parameters((5, 20, 1), (21, 50, 1), (5, 20, 1))

In [ ]:
tester.plot_results()

__Optimization and Backtesting (2004-2014)__ ("In-Sample")

In [ ]:
tester = MACD("AUDEUR=X", EMA_S = 12, EMA_L = 26, signal_mw = 9,
              start = "2004-01-01", end = "2014-12-31", tc = 0)

In [ ]:
tester.optimize_parameters((5, 20, 1), (21, 50, 1), (5, 20, 1))

In [ ]:
tester.plot_results()

__Forward Testing (2015-2020)__ ("Out-Sample")

In [ ]:
forwardtester = MACD("AUDEUR=X", EMA_S = 6, EMA_L = 24, signal_mw = 7,
                     start = "2015-01-01", end = "2020-06-30", tc = 0)

In [ ]:
forwardtester.test_strategy()

In [ ]:
forwardtester.plot_results()

## Alternative Strategies and Interpretations

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import brute
plt.style.use("seaborn")
from MACDBacktester import MACDBacktester as MACD

In [ ]:
tester = MACD("EURUSD=X", EMA_S = 12, EMA_L = 26, signal_mw = 9,
              start = "2004-01-01", end = "2020-06-30", tc = 0)

In [ ]:
alt = tester.data.dropna()
alt

__Main Strategy: MACD Signal Line Crossover__

In [ ]:
alt.loc["2016", ["MACD", "MACD_Signal"]].plot(figsize = (12, 8))
#plt.hlines(y = 0, xmin = alt.index[0], xmax = alt.index[-1], color = "r", label = "Zero")
plt.title("Signal Line Crossover", fontsize = 15)
plt.legend()
plt.show()

__Alternative 1: Zero Crossover__

In [ ]:
alt.loc["2016", "MACD"].plot(figsize = (12, 8))
plt.hlines(y = 0, xmin = alt.index[0], xmax = alt.index[-1], color = "r", label = "Zero")
plt.title("Zero Crossover", fontsize = 15)
plt.legend()
plt.show()

__Alternative 2: Divergence__

Convergence: 
- if __Price__ reaches __higher highs__ -> __MACD__ reaches __higher highs__ (confirmation)
- if __Price__ reaches __lower lows__ -> __MACD__ reaches __lower lows__ (confirmation) 

In [ ]:
alt.loc["03-2013":"06-2013", ["price", "MACD"]].plot(figsize = (12, 8), secondary_y = "MACD")
plt.show()

Divergence: 
- if __Price__ reaches __higher highs__ -> __MACD fails__ to reach higher highs (__Bearish Divergence__)
- if __Price__ reaches __lower lows__ -> __MACD fails__ to reach lower lows (__Bullish Divergence__)

In [ ]:
alt.loc["03-2016":"06-2016", ["price", "MACD"]].plot(figsize = (12, 8), secondary_y = "MACD")
plt.title("Bearish Divergence", fontsize = 15)
plt.show()

In [ ]:
alt.loc["05-2012":"11-2012", ["price", "MACD"]].plot(figsize = (12, 8), secondary_y = "MACD")
plt.title("Bullish Divergence", fontsize = 15)
plt.show()